In [1]:
## Project: 2014 Heart Disease Mortality of 500 Random Counties, based on the Concentration of McDonald's Restaurants
## Further Considerations by Gender, Race/Ethnicity, and Income Level

## Data Cleaning Notebook

In [2]:
#Dependencies
import pandas as pd
import random
import requests
import json
from pprint import pprint


In [3]:
# (1) IRS Data: Finding Income Information by County

In [4]:
#Importing Raw IRS Data
irs_pd = pd.read_csv("Raw Data/IRS Dataset by County.csv")
irs_pd.head()


,STATEFIPS,STATE,COUNTYFIPS,COUNTYNAME,agi_stub,N1,mars1,MARS2,MARS4,PREP,...,N10300,A10300,N85530,A85530,N85300,A85300,N11901,A11901,N11902,A11902
0,1,AL,0,Alabama,1,25120.0,14000.0,9140.0,1110.0,17260.0,...,4530.0,9068.0,70.0,97.0,0.0,0.0,2750.0,3791.0,10210.0,35026.0
1,1,AL,0,Alabama,2,300920.0,223950.0,26770.0,46400.0,156430.0,...,77720.0,45164.0,0.0,0.0,0.0,0.0,23290.0,10777.0,250850.0,296608.0
2,1,AL,0,Alabama,3,549200.0,257890.0,88300.0,194060.0,323470.0,...,312340.0,280080.0,0.0,0.0,0.0,0.0,39900.0,37545.0,494200.0,1668817.0
3,1,AL,0,Alabama,4,491350.0,200730.0,150260.0,125560.0,281320.0,...,397110.0,950375.0,30.0,20.0,0.0,0.0,70960.0,102612.0,413810.0,1177609.0
4,1,AL,0,Alabama,5,259320.0,75790.0,142750.0,34060.0,156510.0,...,250010.0,1316842.0,0.0,0.0,0.0,0.0,62330.0,133274.0,191980.0,538147.0


In [5]:
#Removing state-level data
irs_pd = irs_pd[irs_pd.COUNTYFIPS !=0]

#Combining State and County columns to more easily merge with CDC data
irs_pd["COUNTYNAME"] = irs_pd["COUNTYNAME"] + ", " + irs_pd["STATE"]

#Removing Rows with Missing Data
irs_pd = irs_pd.dropna(how='any')

#Pulling Needed Information
irs_basic = irs_pd[["COUNTYNAME", "agi_stub", "N1", "A00200"]]
irs_basic.head(10)

,COUNTYNAME,agi_stub,N1,A00200
8,"Autauga County, AL",1,210.0,1403.0
9,"Autauga County, AL",2,3350.0,14437.0
10,"Autauga County, AL",3,5620.0,77486.0
11,"Autauga County, AL",4,5410.0,159789.0
12,"Autauga County, AL",5,3420.0,163864.0
13,"Autauga County, AL",6,2450.0,163709.0
14,"Autauga County, AL",7,3000.0,302302.0
15,"Autauga County, AL",8,360.0,61298.0
16,"Baldwin County, AL",1,1710.0,13073.0
17,"Baldwin County, AL",2,13400.0,53151.0


In [6]:
#Renaming Columns
irs_rename = irs_basic.rename(columns={"COUNTYNAME":"County",
                                       "agi_stub":"Adjusted Gross Income",
                                       "N1":"Number of Returns",
                                       "A00200":"Total Salaries & Wages"})

irs_rename.head(10)

,County,Adjusted Gross Income,Number of Returns,Total Salaries & Wages
8,"Autauga County, AL",1,210.0,1403.0
9,"Autauga County, AL",2,3350.0,14437.0
10,"Autauga County, AL",3,5620.0,77486.0
11,"Autauga County, AL",4,5410.0,159789.0
12,"Autauga County, AL",5,3420.0,163864.0
13,"Autauga County, AL",6,2450.0,163709.0
14,"Autauga County, AL",7,3000.0,302302.0
15,"Autauga County, AL",8,360.0,61298.0
16,"Baldwin County, AL",1,1710.0,13073.0
17,"Baldwin County, AL",2,13400.0,53151.0


In [7]:
#Cleaning Adjusted Gross Income
irs_rename.loc[:, "Adjusted Gross Income"] = irs_rename["Adjusted Gross Income"].astype("str")

irs_rename['Adjusted Gross Income'] = irs_rename['Adjusted Gross Income'].replace(
    {'0': 'No AGI Stub',
     '1': 'Under $1',
     '2': '1 - $9,999',
     '3': '10,000 - $24,999',
     '4': '25,000 - $49,999',
     '5': '50,000 - $74,999',
     '6': '75,000 - $99,999',
     '7': '100,000 - $199,999',
     '8': '$200,000 or more'
    })

irs_rename.head(10)

,County,Adjusted Gross Income,Number of Returns,Total Salaries & Wages
8,"Autauga County, AL",Under $1,210.0,1403.0
9,"Autauga County, AL","1 - $9,999",3350.0,14437.0
10,"Autauga County, AL","10,000 - $24,999",5620.0,77486.0
11,"Autauga County, AL","25,000 - $49,999",5410.0,159789.0
12,"Autauga County, AL","50,000 - $74,999",3420.0,163864.0
13,"Autauga County, AL","75,000 - $99,999",2450.0,163709.0
14,"Autauga County, AL","100,000 - $199,999",3000.0,302302.0
15,"Autauga County, AL","$200,000 or more",360.0,61298.0
16,"Baldwin County, AL",Under $1,1710.0,13073.0
17,"Baldwin County, AL","1 - $9,999",13400.0,53151.0


In [8]:
#Exporting Income Brackets per County Information to a CSV file
irs_rename.to_csv("Final_Clean_IRS_Data.csv", index=False, header=True)

In [9]:
# (2) CDC Data: Finding Heart Disease Mortality Rate by County, including Gender and Race/Ethnicity

In [10]:
#Importing Raw CDC Data
cdc_pd = pd.read_csv("Raw Data/CDC Dataset - Heart Disease by County.csv")
cdc_pd.head()

,Year,LocationAbbr,LocationDesc,GeographicLevel,DataSource,Class,Topic,Data_Value,Data_Value_Unit,Data_Value_Type,Data_Value_Footnote_Symbol,Data_Value_Footnote,StratificationCategory1,Stratification1,StratificationCategory2,Stratification2,TopicID,LocationID,Location 1
0,2014,AK,Aleutians East,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,105.3,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2013,"(55.440626, -161.962562)"
1,2014,AK,Aleutians West,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,211.9,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2016,"(52.995403, -170.251538)"
2,2014,AK,Anchorage,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,257.9,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2020,"(61.159049, -149.103905)"
3,2014,AK,Bethel,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,351.6,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",NaN,NaN,Gender,Overall,Race/Ethnicity,Overall,T2,2050,"(60.924483, -159.749655)"
4,2014,AK,Bristol Bay,County,NVSS,Cardiovascular Diseases,Heart Disease Mortality,NaN,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",~,Insufficient Data,Gender,Overall,Race/Ethnicity,Overall,T2,2060,"(58.754192, -156.694709)"


In [11]:
#Removing state-level data
cdc_pd = cdc_pd[cdc_pd.GeographicLevel !="State"]

#Removing Insufficient Data
cdc_pd = cdc_pd[cdc_pd.Data_Value_Footnote_Symbol !="~"]

#Combining State and County columns to more easily merge with IRS data
cdc_pd["LocationDesc"] = cdc_pd["LocationDesc"] + ", " + cdc_pd["LocationAbbr"]

#Pulling Needed Information
cdc_basic = cdc_pd[["LocationAbbr",
                    "LocationDesc",
                    "Data_Value",
                    "Stratification1",
                    "Stratification2",
                    "Location 1"]]
cdc_basic.head()

,LocationAbbr,LocationDesc,Data_Value,Stratification1,Stratification2,Location 1
0,AK,"Aleutians East, AK",105.3,Overall,Overall,"(55.440626, -161.962562)"
1,AK,"Aleutians West, AK",211.9,Overall,Overall,"(52.995403, -170.251538)"
2,AK,"Anchorage, AK",257.9,Overall,Overall,"(61.159049, -149.103905)"
3,AK,"Bethel, AK",351.6,Overall,Overall,"(60.924483, -159.749655)"
5,AK,"Denali, AK",305.5,Overall,Overall,"(63.678399, -149.962076)"


In [12]:
#Renaming Columns
cdc_rename = cdc_basic.rename(columns={"LocationAbbr":"State",
                                       "LocationDesc":"County",
                                       "Data_Value":"Deaths per 100,000",
                                       "Stratification1":"Gender",
                                       "Stratification2":"Race/Ethnicity",
                                       "Location 1": "Geographical Coordinates"})

cdc_rename.head(10)

,State,County,"Deaths per 100,000",Gender,Race/Ethnicity,Geographical Coordinates
0,AK,"Aleutians East, AK",105.3,Overall,Overall,"(55.440626, -161.962562)"
1,AK,"Aleutians West, AK",211.9,Overall,Overall,"(52.995403, -170.251538)"
2,AK,"Anchorage, AK",257.9,Overall,Overall,"(61.159049, -149.103905)"
3,AK,"Bethel, AK",351.6,Overall,Overall,"(60.924483, -159.749655)"
5,AK,"Denali, AK",305.5,Overall,Overall,"(63.678399, -149.962076)"
6,AK,"Dillingham, AK",411.6,Overall,Overall,"(59.803151, -158.181608)"
7,AK,"Fairbanks North Star, AK",305.7,Overall,Overall,"(64.809327, -146.586265)"
8,AK,"Haines, AK",295.7,Overall,Overall,"(59.140721, -135.513878)"
9,AK,"Juneau, AK",295.7,Overall,Overall,"(58.478434, -134.150437)"
10,AK,"Kenai Peninsula, AK",299.4,Overall,Overall,"(60.271254, -151.556972)"


In [13]:
#Dropping Rows with NaN Values 
cdc_rename = cdc_rename.dropna(how='any')
cdc_rename.head(10)

,State,County,"Deaths per 100,000",Gender,Race/Ethnicity,Geographical Coordinates
0,AK,"Aleutians East, AK",105.3,Overall,Overall,"(55.440626, -161.962562)"
1,AK,"Aleutians West, AK",211.9,Overall,Overall,"(52.995403, -170.251538)"
2,AK,"Anchorage, AK",257.9,Overall,Overall,"(61.159049, -149.103905)"
3,AK,"Bethel, AK",351.6,Overall,Overall,"(60.924483, -159.749655)"
5,AK,"Denali, AK",305.5,Overall,Overall,"(63.678399, -149.962076)"
6,AK,"Dillingham, AK",411.6,Overall,Overall,"(59.803151, -158.181608)"
7,AK,"Fairbanks North Star, AK",305.7,Overall,Overall,"(64.809327, -146.586265)"
8,AK,"Haines, AK",295.7,Overall,Overall,"(59.140721, -135.513878)"
9,AK,"Juneau, AK",295.7,Overall,Overall,"(58.478434, -134.150437)"
10,AK,"Kenai Peninsula, AK",299.4,Overall,Overall,"(60.271254, -151.556972)"


In [14]:
#Exporting Mortality Information to a CSV file
cdc_rename.to_csv("Final_Clean_CDC_Data.csv", index=False, header=True)

In [15]:
# (3) Merging the Two Cleaned DataFrames to One and Creating our Sample Population to Test
df_merge = pd.merge(cdc_rename, irs_rename, on="County")
df_merge.head(10)

,State,County,"Deaths per 100,000",Gender,Race/Ethnicity,Geographical Coordinates,Adjusted Gross Income,Number of Returns,Total Salaries & Wages
0,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)",Under $1,100.0,0.0
1,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","1 - $9,999",1160.0,4069.0
2,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","10,000 - $24,999",1270.0,15724.0
3,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","25,000 - $49,999",1680.0,48739.0
4,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","50,000 - $74,999",1080.0,50947.0
5,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","75,000 - $99,999",690.0,42393.0
6,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","100,000 - $199,999",1050.0,98756.0
7,AK,"Ketchikan Gateway, AK",326.8,Overall,Overall,"(55.559161, -131.351892)","$200,000 or more",220.0,27889.0
8,AK,"Ketchikan Gateway, AK",496.9,Male,Overall,"(55.559161, -131.351892)",Under $1,100.0,0.0
9,AK,"Ketchikan Gateway, AK",496.9,Male,Overall,"(55.559161, -131.351892)","1 - $9,999",1160.0,4069.0


In [19]:
#Using the Merged Dataset to Create a List of 500 Random Counties to be used with the Yelp API 

#Determining Unique Counties
unique_counties = df_merge["County"].unique()

#Pulling a Sample of 500 Unique Counties
random.shuffle(unique_counties)
county_pull = unique_counties[:500]
county_pull

array(['DuPage County, IL', 'Hendricks County, IN', 'Polk County, WI',
       'Cuming County, NE', 'Pontotoc County, MS',
       'Montgomery County, VA', 'Crawford County, GA',
       'Chippewa County, MI', 'Randall County, TX',
       'Richardson County, NE', 'Marshall County, MS',
       'Mitchell County, IA', 'Roscommon County, MI', 'Cass County, IL',
       'Clay County, FL', 'Tulare County, CA', 'Dale County, AL',
       'Kimble County, TX', 'Iosco County, MI', 'Phelps County, MO',
       'Monroe County, MS', 'Stonewall County, TX', 'Long County, GA',
       'Anderson County, TN', 'Torrance County, NM',
       'Cleveland County, OK', 'Worth County, MO', 'Carroll County, IL',
       'Greene County, PA', 'Kidder County, ND', 'Hillsdale County, MI',
       'Bon Homme County, SD', 'Burleigh County, ND', 'Kent County, MI',
       'Oneida County, WI', 'Jackson County, SD', 'Vinton County, OH',
       'McCracken County, KY', 'Adams County, NE', 'Windham County, CT',
       'Osceola Count

In [17]:
# (3) Yelp API: Finding number of McDonald's per County

In [20]:
#Creating a Function that Defines Search Parameters for each County

#### WE NEED TO CREATE A SEPARATE CONFIG FILE CONTAINING AN API KEY SO THAT WE'RE NOT HARD-CODING AN API TO THE NOTEBOOK
from api_keys import api_key
url = "https://api.yelp.com/v3/businesses/search"


def get_places(county, page_token):
    params = {
        "Authorization": api_key,
        "location":  county,
        "term": "mcdonalds",
        "limit": 50,
        "categories": "hotdogs",
        }
    
    #using the page token
    if page_token:
        print('using token')
        params['pagetoken'] = page_token

    response = requests.get(url, params=params, headers={"Authorization": f"Bearer {api_key}"})
 
    return response.json()

In [22]:
#creating a loop that uses previous params for each county
from time import sleep

#Defining a new list that will hold all of the restaurant results
rest_list = []

for county in county_pull:

    page_token = None
        
    #using a while loop to countiously loop until False    
    while True:
        
        places_data = get_places(county, page_token)

        #for loop to loop through multiple pages and grab each resturant
        #within each county
        for i in range(0 ,len(places_data['businesses'])):
            rest_list.append({'county': county, 'place_name': places_data["businesses"][i]["name"]})
        
        #using the page token until no more tokens left
        page_token = places_data.get('next_page_token', None)
        print(f'county {county} {page_token}')
        sleep(1)
        if not page_token:
            print('breaking')
            break

county DuPage County, IL None
breaking
county Hendricks County, IN None
breaking
county Polk County, WI None
breaking
county Cuming County, NE None
breaking


KeyboardInterrupt: 

In [44]:
# Turning the resulting list into a dataframe
rest_df = pd.DataFrame(rest_list)
rest_df.head(50)

,county,place_name
0,"Camden County, MO",McDonald's
1,"Camden County, MO",McDonald's
2,"Camden County, MO",McDonald's
3,"Camden County, MO",McDonald's
4,"Camden County, MO",McDonald's
5,"Camden County, MO",McDonald's
6,"Camden County, MO",McDonald's
7,"Camden County, MO",McDonald's
8,"Camden County, MO",Culver's
9,"Camden County, MO",Zaxby's Chicken Fingers & Buffalo Wings


In [45]:
#Eliminating results that are not McDonalds
mcdonalds_df = rest_df.loc[rest_df["place_name"] == "McDonald's",:]

mcdonalds_df.head(50)

,county,place_name
0,"Camden County, MO",McDonald's
1,"Camden County, MO",McDonald's
2,"Camden County, MO",McDonald's
3,"Camden County, MO",McDonald's
4,"Camden County, MO",McDonald's
5,"Camden County, MO",McDonald's
6,"Camden County, MO",McDonald's
7,"Camden County, MO",McDonald's
50,"King County, TX",McDonald's
62,"Tensas Parish, LA",McDonald's


In [46]:
# Creating a dataframe that lists the number of McDonalds per County
mcdonalds_count = mcdonalds_df.groupby(["county"]).count()
mcdonalds_count_df = mcdonalds_count.rename(columns = {"place_name": "Number of Mcdonalds"})
mcdonalds_count_df

,Number of Mcdonalds
county,
"Camden County, MO",8
"King County, TX",1
"Raleigh County, WV",10
"Sullivan County, NH",8
"Tensas Parish, LA",6


In [ ]:
# Exporting the results to a CSV File
mcdonalds_count_df.to_csv("Mcdonalds_per_County.csv", index = True, header = True)